In [1]:
from arcgis.gis import GIS
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools

%load_ext autoreload
%autoreload 2

In [2]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))
gis

GIS @ https://knu2xs.maps.arcgis.com version:6.4

In [3]:
url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [4]:
def process_data(q):
    while True:
        reach_id = q.get()
        
        reach = reach_tools.Reach.get_from_aw(reach_id)
        if reach:
            if reach.has_a_point:
                reach.update_putin_takeout_and_trace()
                reach.publish(gis, lyr_reach_line, lyr_reach_centroid, lyr_reach_points)
                print(reach_id, 'PUBLISHED -', reach.river_name, reach.reach_name)
            else:
                print(reach_id, 'no access coordinates - ', reach.river_name, reach.reach_name)
        else:
            print(reach_id, 'no reach')

In [ ]:
processed_df = lyr_reach_points.query(out_fields='reach_id', return_geometry=False).sdf
unprocessed_reach_id_lst = [val for val in range(1, 100001) if val not in processed_df.reach_id.astype('int64').values]

q = Queue()
processes = [Process(target=process_data, args=(q,)) for i in range(8)]

for p in processes:
    p.start()

for reach_id in unprocessed_reach_id_lst:
    q.put(reach_id)
    
for p in processes:
    p.join()

1217 no access coordinates -  Rio Chama 01\. Lobo Lodge to Chama River Park
689 no access coordinates -  Hoosic First Adams Dam to MA-5
2502 no reach
2570 no reach
1967 no access coordinates -  Mayo, N. Fork Va Route 629 to NC Route 1359
892 no access coordinates -  Seavey Brook Long Lake to E. Machias R.
1098 no access coordinates -  Mayo, S. Fork VA Route 695 to Mayo River along NC Route 1359
2028 no access coordinates -  Adamson, W. Branch Route 100A to Putney
2577 no reach
2576 no reach
3208 no access coordinates -  Swift Creek None
2578 no reach
3118 no access coordinates -  Logan Creek 1\. Upper
2657 no access coordinates -  dsjh galinas bridge to prata landing
3010 no access coordinates -  Meramec Short Bend off Highway 19 to Flamm City Access below Highway 231
3119 no access coordinates -  Logan Creek 2\. Lower
3678 no access coordinates -  Lake Creek (unknown location)
3665 no access coordinates -  Rock Creek Above the Hatchery
3908 no access coordinates -  Gulf Creek Gulf Sta

Process Process-5:
Traceback (most recent call last):
  File "/anaconda3/envs/arcgis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/envs/arcgis/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6b6f06fe2f57>", line 7, in process_data
    if reach.has_a_point:
  File "../src/reach_tools.py", line 391, in has_a_point
    elif self.putin.geometry.type == 'Point' or self.putin.geometry == 'Point':
AttributeError: 'NoneType' object has no attribute 'geometry'


10353 PUBLISHED - Yellow Medicine Hanley Falls to Minnesota River (18.8 miles)
10364 PUBLISHED - Beech Fork 2) KY 150 Fredericktown Bridge to KY 605 Manton Bridge (5 miles)
10365 PUBLISHED - Beech Fork 1) KY 55 Maud Bridge to KY 150 Fredericktown Bridge (9 miles)
10368 PUBLISHED - Buffalo Fork Lower
10371 PUBLISHED - Washburn Run Hockman Rd. to Straight creek/ US 68
10369 PUBLISHED - Snake River Heart River to Lewis River
10366 PUBLISHED - Lewis River Canyon
10367 PUBLISHED - Pacific Creek Lower
10372 PUBLISHED - Hiles Run Myers Hollow Rd. to Straight Creek Rd
10379 no reach
10373 PUBLISHED - Ellis Run Roadside section along Ellis Run Rd.
10381 no reach
10382 no reach
10383 no reach
10384 no reach
10374 PUBLISHED - Snake H) Swan Falls Dam to Walters Ferry (Birds of Prey)
10380 PUBLISHED - Red B) Middle Red, Gresham (PnP)
10375 PUBLISHED - Eagle Creek, West Fork Rt 353 to Rt 763
10376 PUBLISHED - Little Miami 2\. Oregonia to Morrow
10377 PUBLISHED - Cloverlick Creek OH-133 to East Fork 

Process Process-8:
Traceback (most recent call last):
  File "/anaconda3/envs/arcgis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/envs/arcgis/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-6b6f06fe2f57>", line 7, in process_data
    if reach.has_a_point:
  File "../src/reach_tools.py", line 391, in has_a_point
    elif self.putin.geometry.type == 'Point' or self.putin.geometry == 'Point':
AttributeError: 'NoneType' object has no attribute 'geometry'


10994 PUBLISHED - Casselman 1) Salisbury to Rockwood
11004 PUBLISHED - Cow Creek, WF None
11010 no access coordinates -  Potash to Confluence (Meander Canyon) Meander Canyou
11009 PUBLISHED - Popple D) Hwy 101 to Pine River (6.4 miles)
10999 PUBLISHED - Trout Creek Dunkley - Flat Tops Wilderness
11007 PUBLISHED - Mantra Falls Fallview Ave to Taywood Road (PnH)
11014 no access coordinates -  Oak Creek 01\. Upper Oak Creek
11005 PUBLISHED - Eau Galle A) 50th Avenue to Boston Road (7.35 miles)
11006 PUBLISHED - Eau Galle B) Foote Avenue to 770th Avenue (4.3 - 19.3 miles)
11016 no reach
11015 PUBLISHED - Monocacy River 4\. Buckeystown Park to Park Mills Road
11011 PUBLISHED - Buffalo Fork, South Fork Hike-In
11017 PUBLISHED - Monocacy River 3\. Pinecliff Park to Buckeystown Park
11018 PUBLISHED - Monocacy River 2\. Devilbiss Bridge Park to Riverbend Park
11012 PUBLISHED - Buffalo Fork Upper (Hike-In)
11023 no reach
11013 PUBLISHED - Oak Creek 02\. Oak Creek Indian Gardens
11022 PUBLISHED -